In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pickle
import logging

In [3]:
# Set logging level to INFO
logging.basicConfig(level=logging.INFO)

In [4]:
class Message:
    '''Class to keep sender, timestamp, public (To/Cc) and hidden (Bcc) recipients of a message
    
    Must map the Message class fields in parseMaildir.py for deserialization purposes
    '''
    def __init__(self, From, mtime, To, Cc, Bcc):
        self.From = From
        self.mtime = mtime
        self.To = To
        self.Cc = Cc
        self.Bcc = Bcc

In [5]:
# Parse the pickle files generated by parseMaildir.py
parsed_logs_folder = 'Enron/parsing/'
social_graph = pickle.load(open(parsed_logs_folder + "social.pkl", "rb"))
log = pickle.load(open(parsed_logs_folder + "replay_log.pkl", "rb"))

In [6]:
def prep_static_view(social_graph):
    '''Prepare the `userset` and `head_dict` structs for the static view scenario
    
    In the static view scenario, all users update their encryption key at the beginning of time.
    We then simulate how the updated keys get propagated after replaying all messages in
    chronological order. We instantiate the `userset` and `head_dict` structs as follows:
    
      * We include in `userset` the users that we have full access to their sent messages
      * We boostrap the social graph of each user in the userset with her future recipients
      * We create a `head_dict` struct to keep track of the latest known head of a user's friends
    '''
    # Set of users we know the social graph for
    userset = set([])

    # Initial state of the global social graph
    head_dict = {}

    # Initialize the latest known head dictionary
    for email_from, info in social_graph.items():
        userset.add(email_from)
        head_dict[(email_from, email_from)] = 1
        for friend in info['friends']:
            head_dict[(email_from, friend)] = 0

    return userset, head_dict

In [7]:
def eval_head_propagation(head_dict):
    '''Given `head_dict`, count how many entries for the friends of
    the userset users are up-to-date, stale, or not updated at all'''
    updated = 0
    stale = 0
    not_updated = 0
    
    for email_from, info in social_graph.items():
        for friend in info['friends']:
            # If entry value is 0, then user did not learn of her friend's updates
            if head_dict[(email_from, friend)] == 0:
                not_updated += 1
                continue

            # If the friend is not included in the userset, and value is greater than 0,
            # user knows of the head update at the beginning of time
            if (friend, friend) not in head_dict:
                updated += 1
            # Else if friend is included in the userset, we check whether the user knows
            # of her friend's latest head, or an older, stale head
            else:
                if head_dict[(email_from, friend)] == head_dict[(friend, friend)]:
                    updated += 1
                else:
                    stale += 1

    return updated, stale, not_updated

In [8]:
'''Simulate static view of Autocrypt

* Public and private recipients learn of the sender's latest head
'''
print("Simulating the static view scenario of Autocrypt:")

userset, head_dict = prep_static_view(social_graph)

enc_emails = 0
total_sent_emails = 0

for email in log:
    recipients = email.To | email.Cc | email.Bcc - {email.From}
    
    # Count how many emails were sent by users in the userset,
    # and how many of those were encrypted
    if email.From in userset:
        total_sent_emails += 1
        enc_emails += 1

        for recipient in recipients:
            try:
                # If sender does not know of a recipient's head, the email is
                # sent in clear text
                if head_dict[(email.From, recipient)] == 0:
                    raise
            except:
                enc_emails -= 1
                break
    
    # For all recipients, update their dict entry for the sender
    for recipient in recipients:
        if email.From in userset:
            latest_head = head_dict[email.From, email.From]
        else:
            latest_head = 1
        head_dict[(recipient, email.From)] = latest_head

updated, stale, not_updated = eval_head_propagation(head_dict)
print("Userset users know of %s updates of their friends, while %s entries were not updated."
      % (updated + stale, not_updated))
print("%s out of the %s emails sent were encrypted." % (enc_emails, total_sent_emails))

Simulating the static view scenario of Autocrypt:
Userset users know of 134 updates of their friends, while 57729 entries were not updated.
543 out of the 116527 emails sent were encrypted.


In [9]:
'''Simulate static view of ClaimChain with public claims

* Public and private recipients learn of the sender's latest head
'''
print("Simulating the static view scenario of ClaimChain with public claims:")

userset, head_dict = prep_static_view(social_graph)

enc_emails = 0
total_sent_emails = 0

for email in log:
    recipients = email.To | email.Cc | email.Bcc - {email.From}
    
    # Count how many emails were sent by users in the userset,
    # and how many of those were encrypted
    if email.From in userset:
        total_sent_emails += 1
        enc_emails += 1

        for recipient in recipients:
            try:
                # If sender does not know of a recipient's head, the email is
                # sent in clear text
                if head_dict[(email.From, recipient)] == 0:
                    raise
            except:
                enc_emails -= 1
                break
    
    # For all recipients, update their dict entry for the sender
    for recipient in recipients:
        if email.From in userset:
            latest_head = head_dict[email.From, email.From]
        else:
            latest_head = 1
        head_dict[(recipient, email.From)] = latest_head
    
    # Update the social graph entries of the recipients for their friends, if the sender
    # knows of a later head
    if email.From in userset:
        for recipient in recipients & userset:
            for friend in social_graph[email.From]['friends']:
                try:
                    if head_dict[(recipient, friend)] < head_dict[(email.From, friend)]:
                        head_dict[(recipient, friend)] = head_dict[(email.From, friend)]
                except:
                    continue
        

updated, stale, not_updated = eval_head_propagation(head_dict)
print("Userset users know of %s updates of their friends, while %s entries were not updated."
      % (updated + stale, not_updated))
print("%s out of the %s emails sent were encrypted." % (enc_emails, total_sent_emails))

Simulating the static view scenario of ClaimChain with public claims:
Userset users know of 190 updates of their friends, while 57673 entries were not updated.
775 out of the 116527 emails sent were encrypted.
